<a href="https://colab.research.google.com/github/Realmbird/Recreation-Consitutional-AI/blob/main/Reward_model_helpful_Qwen_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U datasets

In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
model_name = "Qwen/Qwen3-8B" # Or any other suitable model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Important: Add a pad token if the tokenizer doesn't have one, especially for decoder models.
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) # accepted and rejceted label
# Resize token embeddings if you added a new pad token
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from trl import RewardTrainer, RewardConfig
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./reward_model_training", # Directory to save checkpoints
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    report_to="none"
)

# Configure RewardTrainer
# The RewardTrainer automatically handles the pairwise comparison loss
trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"], # Use the training split
    eval_dataset=dataset["test"], # Use the test split for evaluation
)

In [ ]:
trainer.train()